In [2]:
import pandas as pd
import env
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import acquire_r
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
import sklearn.model_selection
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import logistic_regression_util
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# import knn_lesson_util as util
import wrangle
# import encode
sns.set()

In [ ]:
df = acquire_r.get_data_from_sql()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

### Looks like no values are null

In [ ]:
df[['churn']].sort_values(by = 'churn')

In [ ]:
#df = df.str.strip()
df['total_charges'] = df['total_charges'].str.strip()

In [ ]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
df.isnull().sum()

In [ ]:
# I am going to drop the 11 'blank' (and then converted to null) values in total_charges column
df = df.dropna()

In [ ]:
# convert total_charges from str to float values:
df.total_charges = df.total_charges.astype(float)

In [ ]:
df.head(2)

In [ ]:
df.churn.value_counts()

In [ ]:
df = df.replace('Yes', 1)
df = df.replace('No', 0)
df = df.replace('No internet service', 2)

In [ ]:
df.head(2)

In [ ]:
train, test = sklearn.model_selection.train_test_split(df, train_size=.8, random_state=42)

In [ ]:
train, validate = sklearn.model_selection.train_test_split(train, train_size=.8, random_state=42)

In [ ]:
train['tenure_years'] = train.tenure/12
train['log_total_charges'] = np.log(train.total_charges)
train.tenure_years = train.tenure_years.apply(np.floor)

validate['tenure_years'] = validate.tenure/12
validate['log_total_charges'] = np.log(validate.total_charges)
validate.tenure_years = validate.tenure_years.apply(np.floor)

test['tenure_years'] = test.tenure/12
test['log_total_charges'] = np.log(test.total_charges)
test.tenure_years = test.tenure_years.apply(np.floor)
train.head(3)

In [ ]:
train, test = sklearn.model_selection.train_test_split(df, train_size=.8, random_state=42)
train.shape

In [ ]:
train['services'] = train.online_security + train.online_backup + train.device_protection + train.tech_support + train.streaming_tv

In [ ]:
train.head(2)

In [ ]:
churn = train[train.churn==1]
no_churn = train[train.churn==0]

In [ ]:
sns.lineplot(x='tenure', y = 'services', data = churn[train.tenure<24][train.internet_service_type_id != 3],label = 'churn')
sns.lineplot(x='tenure', y = 'services', data = no_churn[train.tenure<24][train.internet_service_type_id != 3], label = 'no_churn')
plt.title('Tenure vs no of services')

In [ ]:
a = train[train.churn ==1].groupby('tenure').monthly_charges.agg(['mean', 'std'])

In [ ]:
b = train[train.churn ==0].groupby('tenure').monthly_charges.agg(['mean', 'std'])

In [ ]:
c = pd.concat([pd.DataFrame(a), pd.DataFrame(b)], axis =1)
c.columns = [ 'churn', 'churn_sd', 'no_churn', 'no_churn_sd']
c = c.reset_index()

In [ ]:
sns.lineplot(x = 'tenure',y = 'churn', data = c, label = 'Churned')
sns.lineplot(x = 'tenure', y = 'no_churn', data = c, label = "No Churn")
plt.ylabel(' Average Monthly Charges')

In [ ]:
train.tenure.mean()

In [ ]:
churn.senior_citizen.value_counts()

In [ ]:
no_churn.senior_citizen.value_counts()

In [ ]:
sns.lineplot(x='tenure', y = 'senior_citizen', data = train[train.churn==1], ci =95, label = 'churn')
sns.lineplot(x='tenure', y = 'senior_citizen', data = train[train.churn==0], ci =95, label = 'no_churn')

In [ ]:
sns.lineplot(x='tenure', y = 'dependents', data = train[train.churn==1], ci =95, label = 'churn')
sns.lineplot(x='tenure', y = 'dependents', data = train[train.churn==0], ci =95, label = 'no_churn')

In [ ]:
sns.lineplot(x='tenure', y = 'monthly_charges', data = train[train.churn==1], ci =95, label = 'churn')
sns.lineplot(x='tenure', y = 'monthly_charges', data = train[train.churn==0], ci =95, label = 'no_churn')

In [ ]:
fo = churn[churn.internet_service_type == 'Fiber optic']\
                  [churn.phone_service ==1]
dsl = churn[churn.internet_service_type == 'DSL']\
                  [churn.phone_service ==0]
dsl_ph = churn[churn.internet_service_type == 'DSL']\
                  [churn.phone_service ==1]
ph =churn[churn.internet_service_type == 'None']\
                  [churn.phone_service ==1]

fo0 = no_churn[no_churn.internet_service_type == 'Fiber optic']\
                  [no_churn.phone_service ==1]
dsl0 = no_churn[no_churn.internet_service_type == 'DSL']\
                  [no_churn.phone_service ==0]
dsl_ph0 = no_churn[no_churn.internet_service_type == 'DSL']\
                  [no_churn.phone_service ==1]
ph0 = no_churn[no_churn.internet_service_type == 'None']\
                  [no_churn.phone_service ==1]

In [ ]:
fo.shape, dsl.shape, dsl_ph.shape, ph.shape

In [ ]:
fo0.shape, dsl0.shape, dsl_ph0.shape, ph0.shape

In [ ]:
plt.figure(figsize = (12,9))
sns.lineplot(x='tenure', y = 'monthly_charges', data = fo)
sns.lineplot(x='tenure', y = 'monthly_charges', data = fo0)


sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl)
sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl0)


sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl_ph)
sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl_ph0)


sns.lineplot(x='tenure', y = 'monthly_charges', data = ph)
sns.lineplot(x='tenure', y = 'monthly_charges', data = ph0)

In [ ]:
plt.figure(figsize = (12,9))
sns.lineplot(x='tenure', y = 'monthly_charges', data = fo, label = 'churn', err_style= None)
sns.lineplot(x='tenure', y = 'monthly_charges', data = fo0, label = 'no_churn', err_style = None)
plt.title ('Tenure vs monthly charges for Fiber Optic Service ')

In [ ]:
plt.figure(figsize = (12,9))
sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl,label = 'churn')
sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl0, label = 'no_churn')
plt.title ('Tenure vs monthly charges for DSL Service ')

In [ ]:
plt.figure(figsize = (12,9))
sns.lineplot(x='tenure', y = 'monthly_charges', data = ph, label = 'churn')
sns.lineplot(x='tenure', y = 'monthly_charges', data = ph0, label = 'no_churn')
plt.title ('Tenure vs monthly charges for Phone only Service ')

In [ ]:
sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl_ph, label = 'churn')
sns.lineplot(x='tenure', y = 'monthly_charges', data = dsl_ph0, label = 'no_churn')
plt.title ('Tenure vs monthly charges for DSL + Ph Service ')

In [ ]:
d = pd.crosstab(train[train.internet_service_type=='Fiber optic'].tenure,train[train.internet_service_type=='Fiber optic'].churn ).reset_index()

In [ ]:
sns.lineplot(x= 'tenure', y =d[1], data = d)
sns.lineplot(x= 'tenure', y =d[0], data = d)

In [ ]:
churn.info()

In [ ]:
fo = (pd.DataFrame(churn[train.internet_service_type == 'Fiber optic']\
                  [churn.phone_service ==1].groupby('tenure').monthly_charges.mean())).reset_index()

In [ ]:
dsl_ph = (pd.DataFrame(churn[train.internet_service_type == 'DSL']\
                  [churn.phone_service ==1].groupby('tenure').monthly_charges.mean())).reset_index()

In [ ]:
dsl = (pd.DataFrame(churn[train.internet_service_type == 'DSL']\
                  [churn.phone_service ==0].groupby('tenure').monthly_charges.mean())).reset_index()

In [ ]:
ph = (pd.DataFrame(churn[train.internet_service_type == 'None']\
                  [churn.phone_service ==1].groupby('tenure').monthly_charges.mean())).reset_index()

In [ ]:
fo0 = (pd.DataFrame(no_churn[train.internet_service_type == 'Fiber optic']\
                  [no_churn.phone_service ==1].groupby('tenure').monthly_charges.mean())).reset_index()
dsl_ph0 = (pd.DataFrame(no_churn[train.internet_service_type == 'DSL']\
                  [no_churn.phone_service ==1].groupby('tenure').monthly_charges.mean())).reset_index()
dsl0 = (pd.DataFrame(no_churn[train.internet_service_type == 'DSL']\
                  [no_churn.phone_service ==0].groupby('tenure').monthly_charges.mean())).reset_index()
ph0 = (pd.DataFrame(no_churn[train.internet_service_type == 'None']\
                  [no_churn.phone_service ==1].groupby('tenure').monthly_charges.mean())).reset_index()

In [ ]:
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = fo, label = 'Churned')
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = fo0, label = "No Churn")
plt.ylabel(' Average Monthly Charges')

In [ ]:
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = dsl, label = 'Churned')
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = dsl0, label = "No Churn")
plt.ylabel(' Average Monthly Charges')

In [ ]:
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = dsl_ph, label = 'Churned')
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = dsl_ph0, label = "No Churn")
plt.ylabel(' Average Monthly Charges')

In [ ]:
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = ph, label = 'Churned')
sns.lineplot(x = 'tenure', y = 'monthly_charges', data = ph0, label = "No Churn")
plt.ylabel(' Average Monthly Charges')

In [ ]:
churn.shape, no_churn.shape

In [ ]:
pd.crosstab(churn.internet_service_type, churn.churn, normalize = 'columns')

In [ ]:
pd.crosstab(no_churn.internet_service_type, no_churn.churn, normalize = 'columns')

In [ ]:
test.tenure_years.max()

In [ ]:
train_plot = train[['tenure', 'total_charges', 'monthly_charges', 'log_total_charges','churn']]

In [ ]:
corr = train.corr()
corr

In [ ]:
sns.pairplot(train_plot, vars = train_plot.columns[:-1], hue='churn', plot_kws=dict(alpha=.4, edgecolor='none'))

In [ ]:
train_plot = train[['churn', 'contract_type_id']]

In [ ]:
train.contract_type.value_counts()

In [ ]:
pd.crosstab(train.churn, train.contract_type_id, normalize = 'columns')

### Based on this,  month to month customers are most likely to churn

In [ ]:
pd.crosstab(train.churn, train.payment_type_id, normalize = 'columns')

In [ ]:
t = pd.crosstab(train.tenure_years,train.churn, normalize = 'index')
t

Customers with lower tenure are more likely to churn

 ### Is there a tenure that month-to-month customers are most likely to churn? 1-year contract customers?\



In [ ]:
ten0 = pd.crosstab(train[train.contract_type_id ==1].tenure_years,train[train.contract_type_id ==1].churn, normalize = 'index')
ten0

In [ ]:
ten0.plot(kind='bar', stacked=True)

In [ ]:
ten1 = pd.crosstab(train[train.contract_type_id ==2].tenure_years,train[train.contract_type_id ==2].churn, normalize = 'index')
ten1

In [ ]:
ten1.plot(kind='bar', stacked=True)

###  2-year customers? Do you have any thoughts on what could be going on? (Be sure to state these thoughts not as facts but as untested hypotheses. Unless you test them!).\

In [ ]:
ten2 = pd.crosstab(train[train.contract_type_id ==3].tenure_years, train[train.contract_type_id ==3].churn,  normalize = 'index')
ten2

In [ ]:
ten2.plot(kind='bar', stacked=True)

###  Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers).

In [ ]:
sns.lineplot( x = ten0.index, y = ten0[1], data = ten0, label = 'month to month')
sns.lineplot( x = ten1.index, y = ten1[1], data = ten1, label = 'one year contract')
sns.lineplot( x = ten2.index, y = ten2[1], data = ten2, label = 'two year contract')
sns.lineplot( x = t.index, y = t[1], data = t, label = 'All Customers')
plt.title("% of customers churning")
plt.xlabel('Tenure (years)')
plt.ylabel('% Churn')

### Based on this the month-to month customers are most likely to churn early in their tenure
***

### Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?

In [ ]:
df_plot = pd.crosstab(train.internet_service_type, train.churn)

In [ ]:
df_plot.plot(kind='bar', stacked=True)

### customers with internet type = fibre optics are most likely to churn
***

In [ ]:
df_plot = pd.crosstab(train.phone_service, train.churn)
df_plot1 = pd.crosstab(train.gender, train.churn)

In [ ]:
#plt.figure(figsize=(16, 6))
#plt.subplot(211)
df_plot.plot(kind='bar', stacked=True)
#plt.subplot(212)
df_plot1.plot(kind='bar', stacked=True)

### no significant difference in churn for those with and without phone service
***

In [ ]:
train[train.internet_service_type == 'Fiber optic'].shape

In [ ]:
i_p90 = train[train.internet_service_type == 'Fiber optic'].monthly_charges.quantile(0.9)
i_p90

In [ ]:
train[train.internet_service_type == 'Fiber optic'][train.monthly_charges > i_p90].churn.mean()

In [ ]:
train[train.internet_service_type == 'Fiber optic'][train.monthly_charges < i_p90].churn.mean()

### It is so weird that DSL/Fiber optic customers paying more than 90% percentile are less likely to churn than those paying less than 90 percentile!!
***

In [ ]:
train.churn.sum()

In [ ]:
train[train.contract_type_id == 1][train.tenure_years <= 2]\
[train.monthly_charges > train.monthly_charges.quantile(0.5)].churn.mean()

In [ ]:
train[train.contract_type_id == 1][train.tenure_years <=2]\
[train.monthly_charges < train.monthly_charges.quantile(0.5)].churn.mean()

In [ ]:
exp = train[train.contract_type_id == 1][train.tenure_years < 2][train.churn==0]
exp1 = exp[exp.monthly_charges > exp.monthly_charges.quantile(0.5)]
exp2 = exp[exp.monthly_charges < exp.monthly_charges.quantile(0.5)]

In [ ]:
exp.tenure.count()

In [ ]:
exp.monthly_charges.mean()

In [ ]:
exp.churn.value_counts()

In [ ]:
exp1.churn.mean()

In [ ]:
exp2.churn.mean()

In [ ]:
sns.scatterplot(x= 'tenure', y = 'monthly_charges', data = exp, hue = 'churn' )

### wow! month to month customers with lower tenure and paying more than 50th percentile are way likely to churn compared to those paying less than 50th percentile!. This is specially true where customer has Fiber optic internet service
***

In [ ]:
train[train.contract_type_id == 1][train.monthly_charges > train.monthly_charges.quantile(0.5)].churn.mean()

In [ ]:
train[train.contract_type_id == 1][train.monthly_charges < train.monthly_charges.quantile(0.5)].churn.mean()

### If you are month-to-month customers and paying more than 50 percentile, then they are more likely to churn

In [ ]:
train[train.churn == 1].internet_service_type.value_counts(normalize = True)

1211 churn\
3289 not churn

In [ ]:
train[train.churn == 0].internet_service_type.value_counts(normalize = True)

In [ ]:
train[train.churn == 1].contract_type.value_counts(normalize = True)

In [ ]:
train[train.churn == 0].contract_type.value_counts(normalize = True)

In [ ]:
sns.countplot(x="gender", hue="churn", data=train)

In [ ]:
sns.countplot(x="tenure_years", hue="churn", data=train)

In [ ]:
sns.countplot(x="tenure_years", hue="churn", data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)])

In [ ]:
train[train.monthly_charges > train.monthly_charges.quantile(0.5)].churn.sum()

In [ ]:
train[train.monthly_charges<60].monthly_charges.quantile(0.5)

In [ ]:
train[train.monthly_charges < train.monthly_charges.quantile(0.5)].churn.sum()

In [ ]:
train.head(2)

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(221)
sns.countplot(x="internet_service_type", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])
plt.subplot(222)
sns.countplot(x="senior_citizen", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])

plt.subplot(223)
sns.countplot(x="phone_service", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])

plt.subplot(224)
sns.countplot(x="paperless_billing", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])




In [ ]:
train.device_protection.value_counts()

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(331)
sns.countplot(x="contract_type", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])
plt.subplot(332)
sns.countplot(x="streaming_movies", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])

plt.subplot(333)
sns.countplot(x="online_security", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])

plt.subplot(334)
sns.countplot(x="payment_type", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])

plt.subplot(335)
sns.countplot(x="device_protection", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])
plt.subplot(336)
sns.countplot(x="tech_support", hue="churn",\
              data=train[train.monthly_charges > train.monthly_charges.quantile(0.5)][train.tenure_years <=2])

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(331)
sns.countplot(x="contract_type", hue="churn",\
              data=train)
plt.subplot(332)
sns.countplot(x="senior_citizen", hue="churn",\
              data=train)

plt.subplot(333)
sns.countplot(x="online_security", hue="churn",\
              data=train)

plt.subplot(334)
sns.countplot(x="payment_type", hue="churn",\
              data=train)

plt.subplot(335)
sns.countplot(x="device_protection", hue="churn",\
              data=train)
plt.subplot(336)
sns.countplot(x="tech_support", hue="churn",\
              data=train)

### Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?

In [ ]:
for i in range (1 , 4):
    print(train[train.phone_service == 0][train.internet_service_type_id ==i][train.tenure<2].monthly_charges.count())
    print(train[train.phone_service == 1][train.internet_service_type_id ==i][train.tenure<2].monthly_charges.mean())
    print(train[train.phone_service == 1][train.internet_service_type_id ==i][train.tenure<2].monthly_charges.max())
    print(train[train.phone_service == 1][train.internet_service_type_id ==i][train.tenure<2].monthly_charges.min())

In [ ]:
dsl = train[train.phone_service == 0][train.internet_service_type_id ==1][train.tenure_years <2]


In [ ]:
dsl_ph = train[train.phone_service == 1][train.internet_service_type_id ==1][train.tenure_years <2]

In [ ]:
fibre = train[train.phone_service == 1][train.internet_service_type_id ==2][train.tenure_years <2]

In [ ]:
ph = train[train.phone_service == 1][train.internet_service_type_id ==3][train.tenure_years <2]

In [ ]:
fo = train[train.phone_service == 1][train.internet_service_type_id ==2][train.tenure_years <=2]
fo1 = train[train.phone_service == 0][train.internet_service_type_id ==1][train.tenure_years <=2]
fo.churn.value_counts()

In [ ]:
fo = fo.sort_values(by = 'monthly_charges')
fo1 = fo1.sort_values(by = 'monthly_charges')

In [ ]:
fo['cumsum'] = fo.churn.cumsum()/fo.shape[0]
fo1['cumsum'] = fo1.churn.cumsum()/fo1.shape[0]

In [ ]:
# sns.lineplot(x = 'monthly_charges', y = 'cumsum', data = fo, label = 'phone and fiber optic')
# sns.lineplot(x = 'monthly_charges', y = 'cumsum', data = fo1, label =  'DSL only')
# plt.title('% Cumulative Churn Rate as function of monthly charges')

In [ ]:
fo = train[train.phone_service == 1][train.internet_service_type_id ==2][train.tenure_years <=2]
fo['monthly_bin'] = pd.cut(fo.monthly_charges, bins=10)

In [ ]:
fo.head(2)

In [ ]:
fo = fo.groupby('monthly_bin').churn.mean().reset_index()
fo

In [ ]:
sns.barplot(x = 'monthly_bin', y = 'churn', data = fo)

In [ ]:
train_dsl= train[train.contract_type_id == 1][train.internet_service_type == 'DSL'][train.tenure_years <=2]

In [ ]:
train_fo = train[train.contract_type_id == 1][train.internet_service_type == 'Fiber optic'][train.tenure_years <=2]

In [ ]:
train_dsl.churn.value_counts()

In [ ]:
train_fo.churn.value_counts()

In [ ]:
train.churn.value_counts()

In [ ]:
train_fo['monthly_bin'] = pd.cut(train_fo.monthly_charges, bins =10)

In [ ]:
train_fo = fo.groupby('monthly_bin').churn.mean().reset_index()

In [ ]:
sns.barplot(x = 'monthly_bin', y = 'churn', data = train_fo, orient=45)
plt.xticks(rotation=45)

In [ ]:
#fig, ax = plt.subplots(1, 2, figsize=(15, 3))
train[train.churn ==0].tenure.hist(bins = 60, alpha = 0.5, color= 'purple')
train[train.churn ==1].tenure.hist(bins = 60, alpha = 0.7, color = 'orange')
plt.xlabel('Tenure')
plt.ylabel('no of customers churned')
#sns.kdeplot(train[train.churn ==1].tenure, bw=1.0) 

In [ ]:
plt.hist(train[train.churn ==0].monthly_charges,bins = 60, alpha = 0.7, color = 'purple')
plt.hist(train[train.churn ==1].monthly_charges,bins = 60, alpha = 0.7, color = 'orange')
plt.xlabel('Monthly Charges')
plt.ylabel('no of customers churned')

In [ ]:
plt.hist(train[train.phone_service == 0][train.internet_service_type_id ==1][train.churn ==0].monthly_charges,bins = 30, alpha = 0.7, color = 'purple')
plt.hist(train[train.phone_service == 0][train.internet_service_type_id ==1][train.churn ==1].monthly_charges,bins = 60, alpha = 0.7, color = 'orange')
plt.xlabel('Monthly Charges')
plt.ylabel('no of customers churned')

In [ ]:

f, axes = plt.subplots(1, 2, sharey = True)
sns.kdeplot(train[train.churn ==1].monthly_charges, bw=1.5, label = 'churn', ax=axes[0], color = 'orange')
#sns.kdeplot(train[train.churn ==0].monthly_charges, bw=1.5, label = 'no churn', ax=axes[1], color = 'blue') 

plt.xlabel('monthly charges')
plt.ylabel('no of customers churned')


In [ ]:
sns.kdeplot(train[train.churn ==1].total_charges, bw=1.0) 
plt.xlabel('total_charges')
plt.ylabel('no of customers churned')

In [ ]:
train.info()

In [ ]:
X_train = train[['tenure', 'phone_service', 'tech_support','total_charges', 'internet_service_type_id', 'monthly_charges', 'contract_type_id','payment_type_id']]
y_train = train[['churn']]

X_validate = validate[['tenure','phone_service', 'tech_support', 'total_charges', 'internet_service_type_id', 'monthly_charges', 'contract_type_id','payment_type_id']]

y_validate = validate[['churn']]

In [ ]:
def scale_minmax(train, test, column_list):
    scaler = MinMaxScaler()
    column_list_scaled = [col + '_scaled' for col in column_list]
    train_scaled = pd.DataFrame(scaler.fit_transform(train[column_list]), 
                                columns = column_list_scaled, 
                                index = train.index)
    train = train.join(train_scaled)

    test_scaled = pd.DataFrame(scaler.transform(test[column_list]), 
                                columns = column_list_scaled, 
                                index = test.index)
    test = test.join(test_scaled)

    return train, test

In [ ]:
X_train_scaled, X_validate_scaled = scale_minmax(X_train, X_validate, ['tenure', 'total_charges', 'monthly_charges'])

X_train_scaled = X_train_scaled.drop(columns = ['tenure', 'total_charges', 'monthly_charges'])


X_validate_scaled = X_validate_scaled.drop(columns = ['tenure', 'total_charges', 'monthly_charges'])



In [ ]:
X_train_scaled.head(2)

In [ ]:
logit = LogisticRegression(C=1, random_state = 42)

In [ ]:
logit.fit(X_train_scaled, y_train)

In [ ]:
y_pred = logit.predict(X_train_scaled)

In [ ]:
y_pred_proba = logit.predict_proba(X_train_scaled)
y_pred_proba = pd.DataFrame(y_pred_proba, columns = ['no churn', 'churn'])
y_pred_proba.head()

In [ ]:
logit.classes_

In [ ]:
weights = pd.Series(logit.coef_[0],
index=X_train.columns.values)
weights.sort_values(ascending = False)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
y_pred_proba = logit.predict_proba(X_train_scaled)
y_pred_proba = pd.DataFrame(y_pred_proba, columns = ['no churn', 'churn'])
t = 0.37
yhat = (y_pred_proba > t).astype(int)
print(classification_report(y_train, yhat.churn))

In [ ]:
logistic_regression_util.plot_metrics_by_thresholds(y_train.churn, y_pred_proba.churn)

In [ ]:
clf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=1,
                            n_estimators=1000,
                            max_depth=8, 
                            random_state=42)
#clf = DecisionTreeClassifier(max_depth=3, random_state=42)

In [ ]:
clf.fit(X_train_scaled, y_train)
#y_pred = clf.predict(X_train_scaled)
y_pred = clf.predict(X_validate_scaled)

In [ ]:
pd.DataFrame(classification_report(y_validate, y_pred, output_dict=True))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10, weights='uniform')

In [ ]:
knn.fit(X_train_scaled, y_train)

In [ ]:
y_pred = knn.predict(X_train_scaled)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
# util.plot_k_vs_accuracy(X_train_scaled, y_train, X_validate_scaled, y_validate)

# Lets to encoding and feature engineering 

In [ ]:
df = acquire_r.get_data_from_sql()

In [ ]:
train, validate, test = wrangle.wrangle_telco(df)

In [ ]:
l = ['phone_service', 'contract_type','internet_service_type', 'payment_type','tenure_years']
X_train = encode.encode(train,l)

In [ ]:
column_list = ['log_total_charges', 'monthly_charges']
X_train_scaled = encode.scale_minmax(X_train, column_list)
X_train_scaled.head(2)

In [ ]:
train = train[['senior_citizen', 'phone_service', 'tech_support', 'monthly_charges', 'contract_type', 'internet_service_type','payment_type','tenure_years', 'log_total_charges']]

validate = validate[['senior_citizen', 'phone_service', 'tech_support', 'monthly_charges', 'contract_type', 'internet_service_type','payment_type','tenure_years', 'log_total_charges']]


In [ ]:
train.head(2)

In [ ]:
encoder = sklearn.preprocessing.OneHotEncoder(sparse = False)
encoder.fit(train[['phone_service', 'contract_type','internet_service_type', 'payment_type','tenure_years']])
m = encoder.transform(train[['phone_service', 'contract_type','internet_service_type', 'payment_type','tenure_years']])

In [ ]:
col_name= encoder.get_feature_names(['phone_service', 'contract_type','internet_service_type', 'payment_type','tenure_years'])
train = pd.concat([train, pd.DataFrame(m, columns = col_name,index = train.index)], axis =1)

In [ ]:
col_name

In [ ]:
encoder.fit(validate[['phone_service', 'contract_type','internet_service_type', 'payment_type','tenure_years']])
m = encoder.transform(validate[['phone_service', 'contract_type','internet_service_type', 'payment_type','tenure_years']])

In [ ]:
validate = pd.concat([validate, pd.DataFrame(m, columns = col_name,index = validate.index)], axis =1)

In [ ]:
X_train.head(2)

In [ ]:
X_validate = validate.drop(columns = ['phone_service','phone_service', 'contract_type','internet_service_type', 'payment_type','tenure_years'])
X_validate.head(2)

In [ ]:
X_train_scaled, X_validate_scaled = scale_minmax(X_train, X_validate, ['log_total_charges', 'monthly_charges'])

X_train_scaled = X_train_scaled.drop(columns = ['log_total_charges', 'monthly_charges'])
X_validate_scaled = X_validate_scaled.drop(columns = ['log_total_charges', 'monthly_charges'])

In [ ]:
X_train_scaled

In [ ]:
# logit = LogisticRegression(C=1, random_state = 123, solver='saga')
# logit.fit(X_train_scaled, y_train)
# y_pred = logit.predict(X_train_scaled)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
y_pred_proba = logit.predict_proba(X_validate_scaled)
y_pred_proba = pd.DataFrame(y_pred_proba, columns = ['no churn', 'churn'])
t = 0.33
yhat = (y_pred_proba > t).astype(int)
print(classification_report(y_validate, yhat.churn))

In [ ]:
weights = pd.Series(logit.coef_[0],
index=X_train.columns.values)
weights.sort_values(ascending = False)

In [ ]:
logistic_regression_util.plot_metrics_by_thresholds(y_train.churn, y_pred_proba.churn)

In [ ]:
y_pred = logit.predict(X_validate_scaled)

In [ ]:
print(classification_report(y_validate, y_pred))

In [ ]:
y_pred_proba = logit.predict_proba(X_validate_scaled)
y_pred_proba = pd.DataFrame(y_pred_proba, columns = ['no churn', 'churn'])
t = 0.33
yhat = (y_pred_proba > t).astype(int)
print(classification_report(y_validate, yhat.churn))

In [ ]:
clf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=1,
                            n_estimators=1000,
                            max_depth=8, 
                            random_state=42)

In [ ]:
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_train_scaled)
#y_pred = clf.predict(X_validate_scaled)

In [ ]:
pd.DataFrame(classification_report(y_train, y_pred, output_dict=True))

In [ ]:
y_pred_proba = clf.predict_proba(X_train_scaled)
y_pred_proba = pd.DataFrame(y_pred_proba, columns = ['no churn', 'churn'])
t = 0.33
yhat = (y_pred_proba > t).astype(int)
print(classification_report(y_train, yhat.churn))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=8, weights='distance')

In [ ]:
knn.fit(X_train_scaled, y_train)

In [ ]:
y_pred = knn.predict(X_validate_scaled)

In [ ]:
print(classification_report(y_validate, y_pred))

In [ ]:
# util.plot_k_vs_accuracy(X_train_scaled, y_train, X_validate_scaled, y_validate)

In [ ]:
from sklearn.metrics import auc, roc_curve, roc_auc_score, precision_recall_curve
# plot the ROC curves
plt.figure(figsize=(10,10))



y_pred = logit.predict_proba(X_validate_scaled)[:,1]
fpr, tpr, thresholds = roc_curve(y_validate, y_pred)
plt.plot(fpr, tpr, color='red', lw=2, label='LogisticRegression (area = %0.2f)' % auc(fpr, tpr))
         
y_pred = knn.predict_proba(X_validate_scaled)[:,1]
fpr, tpr, thresholds = roc_curve(y_validate, y_pred)
plt.plot(fpr, tpr, color='yellow', lw=3, label='KNN (area = %0.2f)' % auc(fpr, tpr))


y_pred = clf.predict_proba(X_validate_scaled)[:,1]
fpr, tpr, thresholds = roc_curve(y_validate, y_pred)
plt.plot(fpr, tpr, color='brown', lw=2, label='RandomForest (area = %0.2f)' % auc(fpr, tpr))


plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle= '-', label = 'Random Guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate', fontsize=13)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic', fontsize=17)
plt.legend(loc='lower right', fontsize=13)
plt.show()

In [ ]:
df.monthly_charges.mean()